In [12]:
import pandas as pd
import numpy as np
from libs.location import *
import os
import swifter
import tqdm
import cv2

In [27]:
df = pd.DataFrame({"image_location":os.listdir(BALANCED_TRAINING_IMG_JPG_LOCATION_PATH)},index=None)

In [28]:
df['Target'] = df['image_location'].swifter.apply(lambda col:col[6:7],axis=0)
df['patiendId'] = df['image_location'].swifter.apply(lambda col:col[8:44],axis=0)
df['image_location'] = df['image_location'].swifter.apply(lambda col:f'{BALANCED_TRAINING_IMG_JPG_LOCATION_PATH}/{col}',axis=0)

/home/nitin/miniconda3/envs/ml/lib/python3.9/site-packages/swifter/swifter.py:307: UserWarning: Axis keyword not necessary because applying on a Series.
  warnings.warn("Axis keyword not necessary because applying on a Series.")


Pandas Apply:   0%|          | 0/38710 [00:00<?, ?it/s]

/home/nitin/miniconda3/envs/ml/lib/python3.9/site-packages/swifter/swifter.py:307: UserWarning: Axis keyword not necessary because applying on a Series.
  warnings.warn("Axis keyword not necessary because applying on a Series.")


Pandas Apply:   0%|          | 0/38710 [00:00<?, ?it/s]

/home/nitin/miniconda3/envs/ml/lib/python3.9/site-packages/swifter/swifter.py:307: UserWarning: Axis keyword not necessary because applying on a Series.
  warnings.warn("Axis keyword not necessary because applying on a Series.")


Pandas Apply:   0%|          | 0/38710 [00:00<?, ?it/s]

In [29]:
df.head()

,image_location,Target,patiendId
0,/home/nitin/workspace/notebooks/pneumonia-dete...,1,b3ff1c40-552b-44f9-9cc9-4362905f5eef
1,/home/nitin/workspace/notebooks/pneumonia-dete...,0,cbcb0489-3f9a-47c4-95ab-a7ed364d545a
2,/home/nitin/workspace/notebooks/pneumonia-dete...,0,dff6c869-9e87-49c9-a159-69718967a931
3,/home/nitin/workspace/notebooks/pneumonia-dete...,1,0bed2d9d-8a11-41d9-b312-41f8c93791f7
4,/home/nitin/workspace/notebooks/pneumonia-dete...,0,fefe9ff5-ec06-43bb-a70a-8a7b4b16b3bd


In [30]:
X = []

In [38]:
batch_size = 128

In [41]:
df[:10]

,image_location,Target,patiendId
0,/home/nitin/workspace/notebooks/pneumonia-dete...,1,b3ff1c40-552b-44f9-9cc9-4362905f5eef
1,/home/nitin/workspace/notebooks/pneumonia-dete...,0,cbcb0489-3f9a-47c4-95ab-a7ed364d545a
2,/home/nitin/workspace/notebooks/pneumonia-dete...,0,dff6c869-9e87-49c9-a159-69718967a931
3,/home/nitin/workspace/notebooks/pneumonia-dete...,1,0bed2d9d-8a11-41d9-b312-41f8c93791f7
4,/home/nitin/workspace/notebooks/pneumonia-dete...,0,fefe9ff5-ec06-43bb-a70a-8a7b4b16b3bd
5,/home/nitin/workspace/notebooks/pneumonia-dete...,1,ecc176d8-c3c7-43d8-bcd8-355bbfb84971
6,/home/nitin/workspace/notebooks/pneumonia-dete...,0,ab2981ac-f605-4a8e-a68d-f0e80e424797
7,/home/nitin/workspace/notebooks/pneumonia-dete...,0,efe7f4f8-963f-40be-b2a4-41d335409fda
8,/home/nitin/workspace/notebooks/pneumonia-dete...,0,7ccdb123-2543-4435-a2e3-fe6f2c9f09c5
9,/home/nitin/workspace/notebooks/pneumonia-dete...,0,3e7c788b-3c99-4c44-9877-0ced391d1f75


In [31]:
def create_X_data(batch):
    for index, row in df[batch:].iterrows():
        img_path = row["image_location"]
        img = cv2.imread(img_path)
        if row['Target'] == 0:
            img = cv2.resize(img, (224, 224))
        X.append(img)
        break


In [42]:
for batch in range(1,(len(df)//batch_size)+1):
    print(batch)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [36]:
X = np.array(X).reshape(-1,224, 224, 3)
X.shape

(1, 224, 224, 3)